In [ ]:
"""
IPMU DAQ / Signal-Generator Framework
------------------------------------
- 名前規約: gen / get / start + CamelCase
- 設定: parameters.toml を読み込み
"""

from __future__ import annotations

import sys
import time
import threading
import logging
import logging.handlers as lh
import queue
import h5py
import logging
from pathlib import Path
from typing import Tuple, Iterator
from datetime import datetime

import numpy as np

# ------------------------------------------------------------  config loader
try:
    import tomllib  # Python 3.11+
except ModuleNotFoundError:          # fallback
    import tomli as tomllib

if "__file__" in globals():
    BASE_DIR = Path(__file__).resolve().parent      # スクリプト実行時
else:
    BASE_DIR = Path.cwd()                           # 対話モード／REPL

CONFIG_PATH = BASE_DIR / "parameters.toml"


def _loadConfig(path: Path = CONFIG_PATH) -> dict:
    with path.open("rb") as f:
        return tomllib.load(f)


CONF = _loadConfig()

# Expose convenience aliases
ENC = CONF["encoder"]
IO  = CONF["io"]
GEN = CONF["general"]
GUI = CONF["gui"]
PW  = CONF["power"]
LOG = CONF["logging"]

| #  | 関数名                 | prefix | 役割                   |
| -- | ------------------- | ------ | -------------------- |
| 1  | `genChunkPulse`     | gen    | パルス 1 チャンク生成         |
| 2  | `genChunkSin`       | gen    | 正弦波 1 チャンク生成         |
| 3  | `getPulseDirection` | get    | A/B 立ち上がり→方向判定       |
| 4  | `getPulseCount`     | get    | 方向ログ → パルス数          |
| 5  | `genQuadPulse`      | gen    | 4× 逓倍パルス生成           |
| 6  | `startRoutineIO`    | start  | DEBUG⇆DAQ を切替える本体ループ |
| 7  | `startLog`          | start  | QueueListener を走らせる  |
| 8  | `genCommandFreq`    | gen    | 理想速度コマンド列を生成         |
| 9  | `getPower`          | get    | 三相電力を計算              |
| 11 | `startProcessor`    | start  | 受信データ処理ループ           |
| 12 | `startGui`          | start  | GUI を起動              |
| 13 | `startGuiRefresh`   | start  | GUI の周期更新            |
| 14 | `genTime`           | gen    | 等間隔タイムスタンプのジェネレータ    |
| 15 | **`getSingleDAQ`**  | get    | 単発 DAQ（DEBUG 時のみ）    |


In [ ]:
# ------------------------------------------------------------  utility generators / calculators
def genChunkPulse(
    t: np.ndarray,
    *,
    height: float = ENC["pulse_height"],
    width: float = ENC["pulse_width"],
    duty: float = ENC["pulse_duty"],
    phase: float = ENC["pulse_phase_a"],
) -> np.ndarray:
    """
    Generate a rectangular pulse train for the given time vector *t*.

    Parameters
    ----------
    t : np.ndarray
        Time axis [s].
    height : float
        Pulse amplitude [V].
    width : float
        Pulse period [s]. Must be > 0.
    duty : float
        Duty ratio in (0,1].
    phase : float
        Phase offset [s].
    debug : bool
        If True, emit occasional debug logs.

    Returns
    -------
    np.ndarray (float32)
        Pulse values at each *t*.
    """
    if width <= 0:
        raise ValueError("width must be positive")
    if not (0.0 < duty <= 1.0):
        raise ValueError("duty must be in (0,1]")
    mod = (t + phase) % width
    pulse = np.where(mod < duty * width, height, 0.0).astype(np.float32)
    return pulse



def genChunkSin(
    time: np.ndarray,
    *,
    A: float = PW["amplitude"],
    omega: float = PW["omega"],
    phase: float = PW["phase"],
) -> np.ndarray:
    """
    Generate a sine wave chunk.

    Parameters
    ----------
    time : ndarray[float32]
        Time axis for the sine wave.
    A : float
        Amplitude of the sine wave.
    omega : float
        Angular frequency of the sine wave.
    phase : float
        Phase offset in radians.

    Returns
    -------
    ndarray[float32]
        Sine wave values at the given time points.
    """
    return A*np.sin(omega*time + phase)


def getPulseDirection(
    dA: np.ndarray,
    dB: np.ndarray,
    *,
    threshold: float = ENC["threshold"],
    prev_A: bool | None = None,
    prev_B: bool | None = None,
) -> Tuple[np.ndarray, bool, bool]:
    """Return direction log and the latest A/B level."""
    """
    Detect direction for one block **with perfect edge coverage**.

    Parameters
    ----------
    dA, dB : ndarray[float32]
        Analog levels of phase-A / phase-B for the current block.
    threshold : float
        Logic threshold [V].
    prev_A, prev_B : bool | None
        Logical state of A/B *at the end of the PREVIOUS block*.
        • If None (first block), the function falls back to the
          “self-shift” method used before.

    Returns
    -------
    dir_log : ndarray[int8]
        +1 = CW edge, –1 = CCW edge, 0 = no edge.
    last_A, last_B : bool
        Logical state of A/B at the *end* of this block — feed these
        into the next call to avoid losing the boundary edge.
    """
    # --- current logic level ----------------------------------------
    A = dA > threshold
    B = dB > threshold

    # --- previous sample for XOR -----------------------------------
    if prev_A is None:  # first block → old behaviour
        A_prev = np.concatenate(([A[0]], A[:-1]))
        B_prev = np.concatenate(([B[0]], B[:-1]))
    else:               # use states carried over from last block
        A_prev = np.concatenate(([prev_A], A[:-1]))
        B_prev = np.concatenate(([prev_B], B[:-1]))

    dir_log = (B_prev ^ A).astype(int) - (A_prev ^ B).astype(int)

    return dir_log.astype(np.int8), bool(A[-1]), bool(B[-1])


def getPulseCount(dir_log: np.ndarray) -> int:
    """Count pulses (+/-) from direction log."""
    pass


def genQuadPulse(
    t: np.ndarray,
    dir_log: np.ndarray,
    width: float = ENC["quadpulse_width"],
    height: float = ENC["pulse_height"],
    sampling_rate: int = IO["sample_rate"],
) -> np.ndarray:
    """Generate quadrature-encoded pulse train (×4)."""
    pass


def genCommandFreq(
    t_DC: float = GEN["t_DC"],
    pps: int = GEN["pps"],
    step: int = GEN["step"],
    rst: float = GEN["rst"],
    target_freq: float = GEN["target_freq"],
    proc_interval: float = IO["proc_interval"],
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generate ideal velocity command sequence (time axis, velocity array).
    """
    pass


def getPower(
    time: np.ndarray,
    I_u: np.ndarray, I_v: np.ndarray, I_w: np.ndarray,
    V_u: np.ndarray, V_v: np.ndarray, V_w: np.ndarray,
) -> np.ndarray:
    """Calculate three-phase instantaneous power."""
    pass


def genTime(
    sampling_rate: int = IO["sample_rate"],
) -> Iterator[float]:
    """Yield monotonically increasing timestamps at `1/sampling_rate` steps."""
    pass


def getSingleDAQ(
    tp, *,
    sampling_rate: int = IO["sample_rate"],
    debug: bool = GEN["debug"],
):
    """Single-shot data acquisition for debug / test."""
    pass


# ------------------------------------------------------------  thread / loop starters
def startLog(
    log_q: queue.Queue,
    stop_event: threading.Event,
):
    """Start logging QueueListener; runs until `stop_event` is set."""
    handler = logging.StreamHandler(sys.stdout)
    listener = lh.QueueListener(log_q, handler)
    listener.start()
    stop_event.wait()
    listener.stop()

def genH5Storage(
    run_dir: Path,
    log_data_num: int,
    log_chunk: int,
) -> Tuple[h5py.File, h5py.Dataset]:
    run_dir.mkdir(exist_ok=True)
    timestamp = datetime.now().strftime("%y%m%d%H%M%S")
    h5file = h5py.File(run_dir / f"{timestamp}.h5", "w")
    dset = h5file.create_dataset(
        "log",
        shape=(0, log_data_num),
        maxshape=(None, log_data_num),
        dtype=np.float32,
        chunks=(log_chunk, log_data_num),
        compression="gzip",
    )
    return h5file, dset



class IPMUDaq:
    """
    Top-level controller: reads config, starts / stops all subsystems.
    """

    def __init__(self, config: dict = CONF):
        self.cfg = config
        self.debug = config["general"]["debug"]

        self.h5f, self.dset = genH5Storage(Path("../runs"), LOG["data_num"], LOG["log_chunk"])

        # ---- threading primitives
        self._stop = threading.Event()
        self._log_q: queue.Queue = queue.Queue(maxsize=LOG["log_chunk"])

        # ---- derived constants
        self._derive()

        # ---- thread handles (initialised later)
        self._th_io: threading.Thread | None = None
        self._th_proc: threading.Thread | None = None
        self._th_gui: threading.Thread | None = None
        self._th_log: threading.Thread | None = None

    # -------------------- lifecycle helpers
    def _derive(self) -> None:
        """Compute values derived from config."""
        # e.g. self.samples_proc = int(IO["proc_interval"] * IO["sample_rate"])
        pass

    def startRoutineIO(self) -> None:
        """
        Start the main IO loop:
        - DEBUG == True  -> signal generator mode
        - DEBUG == False -> real DAQ mode
        """
        pass

    def startProcessor(self) -> None:
        """Start data-processing loop (consumer)."""
        pass

    def startGui(self) -> None:
        """Launch GUI in a separate thread or the main thread."""
        pass

    def startGuiRefresh(self) -> None:
        """Periodic GUI refresh loop (runs in GUI thread)."""
        pass

    # -------------------- orchestration
    def run(self) -> None:
        """Spawn all required threads and block until stop."""
        # 1) logging
        self._th_log = threading.Thread(
            target=startLog, args=(self._log_q, self._stop), daemon=True
        )
        self._th_log.start()

        # 2) IO (generator / DAQ)
        self._th_io = threading.Thread(target=self.startRoutineIO, daemon=True)
        self._th_io.start()

        # 3) processor
        self._th_proc = threading.Thread(target=self.startProcessor, daemon=True)
        self._th_proc.start()

        # 4) GUI
        self._th_gui = threading.Thread(target=self.startGui, daemon=True)
        self._th_gui.start()

        try:
            while not self._stop.is_set():
                time.sleep(0.5)
        except KeyboardInterrupt:
            self.stop()

    def stop(self) -> None:
        """Signal all threads to stop and join them."""
        self._stop.set()
        for th in (self._th_io, self._th_proc, self._th_gui, self._th_log):
            if th is not None:
                th.join(timeout=1.0)


# ------------------------------------------------------------  entry point
if __name__ == "__main__":
    daq = IPMUDaq()
    daq.run()
